In [ ]:
import pandas as pd
import numpy as np
from os import listdir
from os import path
import pytz
import glob
from dateutil.parser import isoparse
from datetime import datetime, timedelta
import math
import time
from DirectoryGenerator import DirectoryGenerator

In [ ]:
dirGen = DirectoryGenerator()

In [ ]:
startDate = datetime.strptime(str(input("Start Date (Format: YYYY-MM-DD): ")), "%Y-%m-%d").astimezone(pytz.utc)
startDate

In [ ]:
endDate = datetime.strptime(str(input("End Date (Format: YYYY-MM-DD): ")), "%Y-%m-%d").astimezone(pytz.utc)
endDate

In [ ]:
totalDays = abs(endDate - startDate).days + 1
totalDays

In [ ]:
def parseTimestamp(timestamp):
    try:
        return isoparse(timestamp).astimezone(pytz.utc)
    except:
        return np.nan

In [ ]:
assignmentsDataSubmissionsFilesPath = str(input())

In [ ]:
assignmentsDataSubmissionsFiles = glob.glob(assignmentsDataSubmissionsFilesPath + dirGen.getDelimiter() + '**' + dirGen.getDelimiter() + '*.csv', recursive=True)

In [ ]:
assignmentsDataSubmissionsData = pd.DataFrame()
for assignmentsDataSubmissionsFile in assignmentsDataSubmissionsFiles:
    assignmentsDataSubmissionsData = assignmentsDataSubmissionsData.append(pd.read_csv(assignmentsDataSubmissionsFile), ignore_index = True)
    
assignmentsDataSubmissionsData

In [ ]:
assignmentsDataSubmissionsData = assignmentsDataSubmissionsData[['metadata_event_time', 'metadata_context_id', 'metadata_context_role', 'metadata_user_id', 'body_asset_id']]
assignmentsDataSubmissionsData

In [ ]:
assignmentsDataSubmissionsData['metadata_event_time'] = assignmentsDataSubmissionsData['metadata_event_time'].apply(parseTimestamp)
assignmentsDataSubmissionsData.dropna(subset=['metadata_event_time'], inplace = True)
assignmentsDataSubmissionsData = assignmentsDataSubmissionsData.loc[(assignmentsDataSubmissionsData['metadata_event_time'] >= startDate) & (assignmentsDataSubmissionsData['metadata_event_time'] <= endDate)]
assignmentsDataSubmissionsData

In [ ]:
assignmentsDataSubmissionsData = assignmentsDataSubmissionsData.loc[assignmentsDataSubmissionsData['metadata_context_role'] == 'StudentEnrollment']
assignmentsDataSubmissionsData.drop(['metadata_context_role'], axis = 1, inplace = True)
assignmentsDataSubmissionsData

In [ ]:
courseLessonMaterials = assignmentsDataSubmissionsData.groupby(['metadata_context_id', 'body_asset_id'])['metadata_event_time'].count().to_frame('total').reset_index()
courseLessonMaterials

In [ ]:
courseStudentsFilesPath = str(input())

In [ ]:
courseStudentsFiles = glob.glob(courseStudentsFilesPath + dirGen.getDelimiter() + '**' + dirGen.getDelimiter() + '*.csv', recursive=True)

In [ ]:
courseStudentsData = pd.DataFrame()
for courseStudentsFile in courseStudentsFiles:
    courseStudentsData = courseStudentsData.append(pd.read_csv(courseStudentsFile), ignore_index = True)
    
courseStudentsData

In [ ]:
courseStudentsData['metadata_event_time'] = courseStudentsData['metadata_event_time'].apply(parseTimestamp)
courseStudentsData.dropna(subset=['metadata_event_time'], inplace = True)
courseStudentsData = courseStudentsData.loc[(courseStudentsData['metadata_event_time'] >= startDate) & (courseStudentsData['metadata_event_time'] <= endDate)]
courseStudentsData

In [ ]:
courseStudentsData = courseStudentsData.groupby(['metadata_context_id', 'metadata_user_id'])['metadata_event_time'].count().to_frame('total').reset_index()
courseStudentsData.drop(['total'], axis = 1, inplace = True)
courseStudentsData

In [ ]:
courseStudentAssignmentsComplete = pd.DataFrame()

for course in courseLessonMaterials['metadata_context_id'].unique():
    courseStudentAssignmentsCompleteCourse = pd.DataFrame({'body_asset_id': courseLessonMaterials.loc[courseLessonMaterials['metadata_context_id'] == course]['body_asset_id'].unique()}).merge(pd.DataFrame({'metadata_user_id': courseStudentsData.loc[courseStudentsData['metadata_context_id'] == course]['metadata_user_id'].unique()}), how = 'cross')
    courseStudentAssignmentsCompleteCourse['metadata_context_id'] = course
    courseStudentAssignmentsComplete = courseStudentAssignmentsComplete.append(courseStudentAssignmentsCompleteCourse, ignore_index = True)

courseStudentAssignmentsComplete

In [ ]:
assignmentsDataSubmissionsData = assignmentsDataSubmissionsData.sort_values(by='metadata_event_time').drop_duplicates(subset=['body_asset_id', 'metadata_context_id', 'metadata_user_id'])
assignmentsDataSubmissionsData

In [ ]:
assignmentsDataSubmissionsDataFirstComment = assignmentsDataSubmissionsData.groupby(['metadata_context_id', 'body_asset_id'])['metadata_event_time'].min().to_frame('first_submission_time').reset_index()
assignmentsDataSubmissionsDataFirstComment

In [ ]:
assignmentsDataSubmissionsData = assignmentsDataSubmissionsData.merge(assignmentsDataSubmissionsDataFirstComment, on = ['metadata_context_id', 'body_asset_id'])
assignmentsDataSubmissionsData

In [ ]:
def calculateDelay(df):
    return (time.mktime(df['metadata_event_time'].timetuple()) / 3600) - (time.mktime(df['first_submission_time'].timetuple()) / 3600)

assignmentsDataSubmissionsData['delay_lessons'] = assignmentsDataSubmissionsData.apply(calculateDelay, axis = 1)
assignmentsDataSubmissionsData.drop(['metadata_event_time', 'first_submission_time'], axis = 1, inplace = True)
assignmentsDataSubmissionsData

In [ ]:
assignmentsDataSubmissionsData = courseStudentAssignmentsComplete.merge(assignmentsDataSubmissionsData, how='left', on=['body_asset_id', 'metadata_context_id', 'metadata_user_id'])
assignmentsDataSubmissionsData

In [ ]:
assignmentsDataSubmissionsData = assignmentsDataSubmissionsData.fillna(((time.mktime((endDate + timedelta(seconds = 1)).timetuple()) / 3600) - (time.mktime(startDate.timetuple()) / 3600)))
assignmentsDataSubmissionsData

In [ ]:
assignmentsDataSubmissionsData = assignmentsDataSubmissionsData.groupby(['metadata_context_id', 'metadata_user_id'])['delay_lessons'].mean().to_frame('ave_delay').reset_index()
assignmentsDataSubmissionsData

In [ ]:
assignmentsDataSubmissionsData.rename(columns={"ave_delay": "ave_delay_L"}).to_csv("LessonsDelay_Intercession_2022.csv", header = True, index = False)